In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import statsmodels.api as sm
import matplotlib.patches as mpatches

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import SilhouetteVisualizer

from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from collections import Counter
from kneebow.rotor import Rotor

pd.set_option('display.max_columns', None)
path = "../../data/"

### From the scaling law...

* Thermal energy confinement time: $\tau_{E,th} = W_{th}/P_{l,th}$ [s]
    - Thermal stored energy: $W_{th}$
* Intercept of the regression: $\alpha_0$    
* Plasma current: $I_P$ [MA]
* Vacuum toroidal magnetic field: $B_t$ [T]
* Central line-averaged electron density: $\bar{n}_e$ [$\cdot$10$^{19}$ m $^{-3}$] 
* Thermal power lost due to the transport through the LCFS: $P_{l,th}$ [MW]
* Major radius: $R_{geo}$ [m]
* Elongation of the last-closed flux surface: $\kappa = V/(2\pi R_{geo}\pi a^2)$ 
    - Plasma volume inside the LCFS: V [m$^{3}$]
    - Minor radius: $a$ [m] 
* Inverse aspect ration: $\epsilon = a/R_{geo}$ 
    - large $\epsilon$ = spherical tokamak (NSTX and MAST)
* Effective atomic mass of the plasma $M_{eff}$

$$
    \tau_{E,th} = \alpha_0\cdot I_P^{\alpha_I}\cdot B^{\alpha_B}_t\cdot \bar{n}^{\alpha_n}_e  \cdot P^{\alpha_P}_{l,th}\cdot R^{\alpha_R}_{geo}\cdot \kappa^{\alpha_\kappa}_a\cdot \epsilon^{\alpha_\epsilon}\cdot M^{\alpha_M}_{eff}
$$

It is possible to express $\tau_{E,th}$ in terms of four dimensional variables $\tau_{E,th} = \tau_{E,th}(\hat{n}, \hat{T}, R, B)$ only ; thus, $\omega_{c}\tau_{E,th}$ is expected to scale with $(\rho_*, \nu_*, \beta)$. These analyses are referred as Kadomtsev's transformation and Kadomtsev's constraint.

[[Y. Sarazin et al, 2019]](https://iopscience.iop.org/article/10.1088/1741-4326/ab48a5/meta).


Reading [HDB5V2.3](https://dataspace.princeton.edu/handle/88435/dsp01m900nx49h) database.

In [3]:
DB5 = pd.read_excel("../../data/HDB5V2.3.xlsx")

In [4]:
# Reading data and preparing it

DB5_ = pd.read_csv(path+"DB5.csv") #From DB5 dataset | log data
#subset_ids = pd.read_csv(path+"R_ids_alpha_0.6556.csv")
#DB5_ = (DB5_[DB5_.id.isin(subset_ids.id.values)]).reset_index(drop=True)

In [5]:
DB5_

,ind,id,PHASE,TOK,IP,BT,NEL,PLTH,RGEO,KAREA,EPS,MEFF,TAUTH,DATE,SHOT,TIME,Q95,ZEFF,AMIN,VOL,POHM,PNBI,DWDIA,DWMHD,PICRH,PECRH,PL,PFLOSS,TAV,LCOULOMB,QCYL5,TAUBOHM,RHOSTAR,BETASTAR,NUSTAR,OMEGACYCL
0,12405,HDULEH,HGELM,ASDEX,0.2959,2.205,3.789,1.8700,1.6940,0.9748,0.242090,1.5,0.05100,19820622,5980,1.205,4.563,NaN,0.4101,5.4820,68660.0,2599000.0,212000.0,212000.0,0.0,0.0,2456000.0,585700.0,1086.986716,15.406717,3.606992,0.074970,0.006430,0.601125,0.223030,1.470000
1,12406,NAC6N1,HGELM,ASDEX,0.2952,2.205,3.734,2.0240,1.6840,0.9724,0.244240,1.5,0.04902,19820622,5980,1.224,4.671,NaN,0.4113,5.4680,22960.0,2598000.0,0.0,0.0,0.0,0.0,2621000.0,596700.0,1150.578521,15.470883,3.647604,0.072059,0.006596,0.627056,0.195418,1.470000
2,12411,U2T1C7,HSELM,ASDEX,0.2971,2.205,3.410,1.1320,1.6930,0.9895,0.238807,1.5,0.06375,19820622,5982,1.188,4.275,NaN,0.4043,5.4050,0.0,2590000.0,815400.0,942200.0,0.0,0.0,1690000.0,558000.0,927.006370,15.300207,3.544311,0.093713,0.006023,0.461374,0.274718,1.470000
3,12412,422XQB,HGELM,ASDEX,0.2959,2.205,3.775,1.3760,1.6930,0.9744,0.242351,1.5,0.06991,19820622,5982,1.216,4.574,NaN,0.4103,5.4820,31630.0,2599000.0,683900.0,662500.0,0.0,0.0,1961000.0,584900.0,1100.512996,15.420935,3.611224,0.102768,0.006467,0.606357,0.216753,1.470000
4,12413,WZ9FED,HGELM,ASDEX,0.2942,2.204,3.847,2.0330,1.6850,0.9691,0.244926,1.5,0.05151,19820622,5982,1.244,4.758,NaN,0.4127,5.4900,37030.0,2600000.0,0.0,0.0,0.0,0.0,2637000.0,604000.0,1173.621999,15.475806,3.668909,0.075685,0.006642,0.659569,0.193993,1.469333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6245,24961,NGC85H,HSELM,TDEV,-0.2198,1.963,4.919,0.6145,0.8274,1.1080,0.259971,2.0,0.01666,19980407,33119,0.800,3.203,NaN,0.2151,0.8371,102100.0,0.0,-1906.0,-1906.0,0.0,510500.0,614500.0,0.0,588.737748,14.663031,2.793295,0.016352,0.011702,0.533324,0.319289,0.981500
6246,24963,ID1IP4,HSELM,TDEV,-0.2199,1.952,5.109,0.6358,0.8356,1.1010,0.258018,2.0,0.01710,19980408,33155,0.800,3.143,NaN,0.2156,0.8439,125200.0,0.0,401.3,401.3,0.0,511000.0,635800.0,0.0,596.868618,14.657798,2.757587,0.016690,0.011822,0.567921,0.325224,0.976000
6247,24964,K19A5H,HSELM,TDEV,-0.2198,1.951,5.337,0.3762,0.8350,1.1030,0.257844,2.0,0.02543,19980408,33171,0.800,3.131,NaN,0.2153,0.8426,167500.0,0.0,-1447.0,-1447.0,0.0,207200.0,376200.0,0.0,503.655066,14.466163,2.757718,0.024807,0.010880,0.501128,0.471050,0.975500
6248,26411,D26CA1,HSELM,TFTR,0.9794,4.795,3.500,5.1120,2.4540,0.9997,0.327343,2.0,0.17310,19900124,45980,4.600,8.050,NaN,0.8033,31.2500,-732600.0,10310000.0,162800.0,162800.0,0.0,0.0,9410000.0,4297000.0,1916.017316,16.013226,6.010681,0.415007,0.002314,0.206978,0.105809,2.397500


In [6]:
#DB5["TAUTH"] = DB5.TAUTH.apply(lambda x: round(x,3)) 
db5 = DB5[["TOK","DATE","SHOT","TIME"]]. applymap(str) 
keyDB5 = (db5["TOK"] + db5["DATE"] + db5["SHOT"] + db5["TIME"] )
DB5["key"] = keyDB5

db5_ = DB5_[["TOK","DATE","SHOT","TIME"]]. applymap(str) 
keyDB5_ = (db5_["TOK"] + db5_["DATE"] + db5_["SHOT"] + db5_["TIME"] )
DB5_["key"] = keyDB5_

len( keyDB5.unique() ), len( keyDB5_.unique() )

(14153, 6250)

In [7]:
len(DB5), len(DB5_)

(14153, 6250)

In [8]:
# Complete columns for shots of interest
complete_data = DB5[DB5.key.isin(DB5_.key)]

features = ['NEL','TAUTH','BT', 'TEV', 'TE0', 'TE0TSC']

In [9]:
DB5_["PLTH"]

0       1.8700
1       2.0240
2       1.1320
3       1.3760
4       2.0330
         ...  
6245    0.6145
6246    0.6358
6247    0.3762
6248    5.1120
6249    4.9930
Name: PLTH, Length: 6250, dtype: float64

In [10]:
DB5["PLTH"]

0        315400.0
1        325500.0
2        299200.0
3             NaN
4        752300.0
           ...   
14148    120700.0
14149    121000.0
14150    142100.0
14151    125000.0
14152    144900.0
Name: PLTH, Length: 14153, dtype: float64

In [11]:
complete_data

,TOK,TOK_ID,DIVNAME,LCUPDATE,DATE,SHOT,TIME,TIME_ID,T1,T2,AUXHEAT,PHASE,HYBRID,ITB,ITBTYPE,ELMTYPE,ELMFREQ,ELMMAX,ELMDUR,ELMINT,OLTIME,LHTIME,TPI,ISEQ,MEFF,PGASA,PGASZ,BGASA,BGASZ,BGASA2,BGASZ2,PELLET,FUELRATE,XGASZ,XGASA,CONFIG,RGEO,RMAG,AMIN,KAPPA,KAPPAA,KAREA,DELTA,DELTAU,DELTAL,INDENT,AREA,VOL,SURFFORM,SEPLIM,XPLIM,WALMAT,DIVMAT,LIMMAT,EVAP,DALFMP,DALFDV,IGRADB,BT,IEML,PREMAG,IP,VSURF,Q95,SH95,BEILI2,BEIMHD,BEPMHD,BETMHD,BEPDIA,BMHDMDIA,TAUCR,FBS,RHOQ2,RHOINV,NEL,NELFORM,DNELDT,NEV,NE0,NE0TSC,ZEFF,ZEFFNEO,PRAD,POHM,ENBI,PINJ,BSOURCE,PINJ2,BSOURCE2,COCTR,PNBI,PFLOSS,ECHFREQ,ECHMODE,ECHLOC,PECRHC,PECRH,ICFREQ,ICSCHEME,ICANTEN,PICRHC,PICRH,PALPHA,DWDIA,DWDIAPAR,DWMHD,DWHC,TEV,TE0,TE0TSC,TIV,TI0,TICX0,WDIA,WMHD,WKIN,WEKIN,WIKIN,WROT,WFPER,WFPAR,WFFORM,WFANI,WFICRH,WFICRHP,WFICFORM,ICFORM,WFANIIC,TAUDIA,TAUMHD,TAUTH1,TAUTH2,WTOT,WTH,PL,PLTH,TAUTOT],TAUTH,TAUC92,TAUC93,H89,HITER96L,H93,HITER92Y,HEPS97,HIPB98Y,HIPB98Y1,HIPB98Y2,HIPB98Y3,HIPB98Y4,OMGAIMP0,OMGAIMPH,OMGAM0,OMGAMH,SPIN,TORQ,TORQBM,TORQIN,VTOR0,VTORV,VTORIMP,STANDARD,SELDB1,SELDB2,SELDB2X,IAEA92,DB2P5,DB2P8,DB3IS,DB3V5,IAE2000N,IAE2000X,HMWS2003,IAE2004S,IAE2004I,DB3DONLY,HMWS2005,OJK2006,SELDB3,SELDB3X,SELDB4,AAREA,STDDB4V5,NESEP,NESOL,PMAIN,PDIV,GP_MAIN,GP_DIV,SELDB5,STD3,key
5,ASDEX,1,DV-IPRE,19940125,19820622,5980,1.205,1205,1.203,1.207,NB,HGELM,UNKNOWN,UNKNOWN,NaN,UNKNOWN,NaN,NaN,NaN,NaN,1.110,1.166,6.0,NONE,1.5,2.0,1,1,1,0,0,NONE,,0,0.0,DN,1.6940,1.8030,0.4101,0.9752,0.975092,0.9748,0.0000,0.0000,0.00000,0.0,0.5152,5.4820,27.088080,0.15640,NaN,SS,TI2,NONE,NONE,NaN,NaN,NaN,2.205,0.0,NaN,295900.0,0.23200,4.563,NaN,2.649,2.0190,2.019,0.009265,1.6670,0.4481,5.423056,NaN,NaN,0.277121,37890000000000000000,0.0,-19590000000000000000,30170000000000000000,51000000000000000000,,NaN,NaN,0.0,68660.0,42000.00,2620000.0,404020.0,0.0,0.0,1.0000,2599000.0,585700.0,0.000000e+00,NONE,NONE,0.0,0.0,0.0,NONE,NONE,0.0,0.0,0.0,212000.0,620500.0,212000.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,116400.0,141100.0,NaN,NaN,NaN,NaN,NaN,NaN,24260.0,0.5780,0.0,0.0,0.0,0.0,0.0,0.04746,0.05753,NaN,0.05100,119600.0,95370.0,2456000.0,1870000.0,0.04871,0.05100,0.6667,0.6667,1.612,1.413,0.9272,1.103,1.126,1.150,1.119,1.132,1.130,1.0110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,111.0,1.111111e+09,1.111111e+09,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.111111e+09,111100.0,0.0,NaN,NaN,,,NaN,NaN,NaN,NaN,1,1.0,ASDEX1982062259801.205
6,ASDEX,1,DV-IPRE,19940125,19820622,5980,1.224,1224,1.222,1.226,NB,HGELM,UNKNOWN,UNKNOWN,NaN,UNKNOWN,NaN,NaN,NaN,NaN,1.110,1.166,7.0,NONE,1.5,2.0,1,1,1,0,0,NONE,,0,0.0,DN,1.6840,1.7970,0.4113,0.9724,0.972233,0.9724,0.0000,0.0000,0.00000,0.0,0.5167,5.4680,26.969199,0.16430,NaN,SS,TI2,NONE,NONE,NaN,NaN,NaN,2.205,0.0,NaN,295200.0,0.07778,4.671,NaN,2.720,2.0900,2.090,0.009540,1.7510,0.4348,16.042211,NaN,NaN,0.273998,37340000000000000000,0.0,-44680000000000000000,29400000000000000000,52000000000000000000,,NaN,NaN,0.0,22960.0,42000.00,2620000.0,404020.0,0.0,0.0,1.0000,2598000.0,596700.0,0.000000e+00,NONE,NONE,0.0,0.0,0.0,NONE,NONE,0.0,0.0,0.0,0.0,-485.4,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,121100.0,144500.0,NaN,NaN,NaN,NaN,NaN,NaN,25220.0,0.5782,0.0,0.0,0.0,0.0,0.0,0.04628,0.05522,NaN,0.04902,124400.0,99230.0,2621000.0,2024000.0,0.04748,0.04902,0.6667,0.6667,1.641,1.470,0.9572,1.139,1.165,1.185,1.152,1.169,1.167,1.0440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,111.0,1.111111e+09,1.111111e+09,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.111111e+09,111100.0,0.0,NaN,NaN,,,NaN,NaN,NaN,NaN,1,1.0,ASDEX1982062259801.224
11,ASDEX,1,DV-IPRE,19940125,19820622,5982,1.188,1188,1.186,1.190,NB,HSELM,UNKNOWN,UNKNOWN,NaN,UNKNOWN,NaN,NaN,NaN,NaN,1.115,1.165,5.0,NONE,1.5,2.0,1,1,1,0,0,NONE,,0,0.0,DN,1.6930,1.7870,0.4043,0.9895,0.989248,0.9895,0.0000,0.0000,0.00000,0.0,0.5080,5.4050,26.880691,0.16230,NaN,SS,TI2,NONE,NONE,NaN,NaN,NaN,2.205,0.0,NaN,297100.0,-0.05525,4.275,NaN,2.289,1.6600,1.660,0.007679,1.3130,0.4427,22.850781,NaN,NaN,0.282055,34100000000000000000,0.0,1926000000000000000

In [12]:
# Standard Scaler
X = complete_data[features]
X = StandardScaler().fit_transform(X)
data = pd.DataFrame(X, columns=features)
data["TOK"] = complete_data["TOK"]

In [13]:
complete_data[features]

,NEL,TAUTH,BT,TEV,TE0,TE0TSC
5,37890000000000000000,0.05100,2.205,NaN,NaN,NaN
6,37340000000000000000,0.04902,2.205,NaN,NaN,NaN
11,34100000000000000000,0.06375,2.205,NaN,NaN,NaN
12,37750000000000000000,0.06991,2.205,NaN,NaN,NaN
13,38470000000000000000,0.05151,2.204,NaN,NaN,NaN
...,...,...,...,...,...,...
12561,49190000000000000000,0.01666,1.963,880.5,3105.0,2678.0
12563,51090000000000000000,0.01710,1.952,864.5,2983.0,NaN
12564,53370000000000000000,0.02543,1.951,689.6,1928.0,1618.0
14011,35000000000000000000,0.17310,4.795,1797.0,4517.0,NaN


In [14]:
len(data), len(complete_data)

(4624, 4624)

In [15]:
### Computing the optimal epsilon

#%matplotlib notebook

NNeighbours = 2*X.shape[-1]

# getting the average distance
neighbors = NearestNeighbors(n_neighbors = NNeighbours)
neighbors_fit = neighbors.fit(X)
distances, indices = neighbors_fit.kneighbors(X)
distances = np.sort(distances, axis=0)
distances = distances[:,1]

x = list(range(len(distances)))

plt.figure(figsize=(7,4))
plt.scatter(x, distances, s=1)

data_elbow = (pd.DataFrame([x, distances]).T).values.tolist()
rotor = Rotor()
rotor.fit_rotate( data_elbow )
elbow_index = rotor.get_elbow_index()
plt.axvline(elbow_index, c="k")
#plt.axvline(1204, c="r")

#plt.axis([900, max(x), min(distances), max(distances)])

plt.ylabel("Distance")
plt.xlabel("Object")
plt.grid(True, alpha=0.5);

ValueError: Input X contains NaN.
NearestNeighbors does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# Label -1 means noise
DB = DBSCAN(eps = distances[elbow_index], min_samples=NNeighbours ).fit(X)
labels = DB.labels_
data.insert(loc=0, column="labels", value=labels)
Counter(labels)

In [ ]:
labels_unique = list(range(-1, len(Counter(labels)) -1) )

TOKs_per_lbl = [0]*len(labels_unique)
for i,lbl in enumerate(labels_unique):
    df = data[data.labels.isin([lbl])]
    TOKs_per_lbl[i] = df.TOK.unique()

In [ ]:
# *** Unique Tokamaks found per label 
# *** This will allow to discern wether the clustering happened in obvious parameters
# Index of the df is the label / columns names can be ignored
pd.DataFrame(TOKs_per_lbl, index=labels_unique)

In [ ]:
data_cl = data[~data.labels.isin([-1])][["labels"]+features]
data_cl_mean = data_cl.groupby("labels").mean().T
data_cl_mean.columns = "cluster_"+data_cl_mean.columns.astype(str)
print("Mean values of each feature per cluster")
data_cl_mean

In [ ]:
import plotly.graph_objects as go

categories = features
fig = go.Figure()

for i in range(len(Counter(labels))-1):

    fig.add_trace(go.Scatterpolar(
          r=data_cl_mean[f"cluster_{i}"].values,
          theta=categories,
          fill='toself',
          name=f'Cluster {i}'
    ))



fig.show()